In [2]:
from gurobipy import GRB
import gurobipy as gb

In [3]:
# The parameters
oat_yield = 4.25
maize_yield = 3.0
soybean_yield = 20.0

In [4]:
# Number of options
CROPS = 3
PURCHASED = 2
SOLD = 4

In [5]:
# Selling prices
sell = [220, 260, 55, 26]
purchase = [264, 312]

In [6]:
# Create a new optimization model to maximize profit
model = gb.Model("Farming Problem")

Restricted license - for non-production use only - expires 2025-11-24


In [7]:
# Construct the decision variables.
x = model.addVars(3, lb=0, vtype=GRB.CONTINUOUS, name="Crops")
w = model.addVars(4, lb=0, vtype=GRB.CONTINUOUS, name="Sold")
y = model.addVars(2, lb=0, vtype=GRB.CONTINUOUS, name="Purchased")

In [8]:
# Objective Function
model.setObjective(gb.quicksum(w[i]*sell[i] for i in range(SOLD)) - gb.quicksum(y[i]*purchase[i] for i in range(PURCHASED)), GRB.MAXIMIZE)

In [9]:
# Land capacity constraints 
land_constraint = model.addConstr(x[0] + x[1] + x[2] <= 500, "Land Capacity")

# Cattle feed constraints (oats)
cattle_constraint = model.addConstr(oat_yield*x[0] + y[0] - w[0] >= 200, "Oats")

# Cattle feed constraints (Maize)
oat_constraint = model.addConstr(maize_yield*x[1] + y[1] - w[1] >= 260, "Oats")

# Quota constraints (Soybean)
model.addConstr(w[2] <= 7000, "Quota")
soy_constraint = model.addConstr(w[2] + w[3] == soybean_yield*x[2], "Soybean")

In [10]:
# Solve our model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 5 rows, 9 columns and 13 nonzeros
Model fingerprint: 0x34930c79
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e+01, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 7e+03]
Presolve removed 2 rows and 2 columns
Presolve time: 0.01s
Presolved: 3 rows, 7 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.6800000e+33   2.000000e+30   7.680000e+03      0s
       4    4.0021667e+05   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.002166667e+05


In [11]:
# Append the objective function value
print("The optimal solution: ", model.objVal)

The optimal solution:  400216.6666666667


In [12]:
# Check if the optimization was successful
if model.status == gb.GRB.OPTIMAL:
    # Print the sensitivity analysis for the amount sold
    print("Optimal Amount Sold:")
    print(f"{'Oats'} = {w[0].x, w[0].RC, sell[0], w[0].SAObjUp, w[0].SAObjLow}")
    print(f"{'Maize'} = {w[1].x, w[1].RC, sell[1], w[1].SAObjUp, w[1].SAObjLow}")
    print(f"{'Soybean'} = {w[2].x, w[2].RC, sell[2], w[2].SAObjUp, w[2].SAObjLow}")
    print(f"{'Soybean'} = {w[3].x, w[3].RC, sell[3], w[3].SAObjUp, w[3].SAObjLow}")
else:
    print("Optimization was not successful.")

Optimal Amount Sold:
Oats = (69.16666666666669, 0.0, 220, 220.23529411764707, 183.5294117647059)
Maize = (0.0, -51.66666666666663, 260, 311.66666666666663, -inf)
Soybean = (7000.0, 0.0, 55, inf, 46.75)
Soybean = (0.0, -20.75, 26, 46.75, -inf)


In [13]:
# Print sensitivity information
print("")
print(f"Sensitivity Information for Land Capacity Constraint {land_constraint.pi:.2f}:")
print("(LHS, RHS, Slack): ", (model.getRow(land_constraint).getValue(), land_constraint.RHS, land_constraint.slack))
print("Shadow Price: ", land_constraint.pi)
print("Range of Feasibility: ", (land_constraint.SARHSUp, land_constraint.SARHSLow))


Sensitivity Information for Land Capacity Constraint 935.00:
(LHS, RHS, Slack):  (500.0, 500.0, 0.0)
Shadow Price:  935.0
Range of Feasibility:  (inf, 483.72549019607845)
